In [1]:
import sys
import os
# Add the parent directory to the Python path

# Change the current working directory to /app
os.chdir('/app')
from dotenv import load_dotenv
load_dotenv(dotenv_path='config/.env')
database_path = os.getenv('DATABASE_PATH')
from utils.db.fetch import fetch_entries
print(f'DATABASE_PATH: {database_path}')
market_name = 'crypto_binance'
timeframe='1m'
pair='BTC'

DATABASE_PATH: database/db/ohlcv_data.db


In [2]:
from finstore.finstore import Finstore
finstore = Finstore(market_name=market_name, timeframe=timeframe, enable_append=True)

Len items in 'database/finstore/market_name=crypto_binance/timeframe=1m': 257


In [18]:
df = finstore.stream.fetch_trade_data(symbol='ETHBTC')
df.tail(10)

,timestamp,open,high,low,close,volume,buy_volume
77,1734196560000,0.03823,0.03824,0.03823,0.03823,12.3095,8.5633
78,1734196620000,0.03822,0.03822,0.03820,0.03821,8.9560,4.3406
79,1734196680000,0.03821,0.03822,0.03820,0.03822,26.8177,18.7261
80,1734196740000,0.03822,0.03822,0.03821,0.03821,6.5699,5.3334
81,1734196800000,0.03821,0.03823,0.03821,0.03823,9.8050,7.8050
82,1734196860000,0.03822,0.03822,0.03821,0.03821,2.7007,0.0981
83,1734196920000,0.03820,0.03820,0.03818,0.03819,21.8104,2.4623
84,1734196980000,0.03820,0.03820,0.03818,0.03818,8.3900,4.4939
85,1734197040000,0.03819,0.03819,0.03818,0.03819,4.0837,4.0717
86,1734197100000,0.03819,0.03821,0.03819,0.03820,13.9831,12.9303


In [18]:
df = finstore.stream.fetch_trade_data(symbol='AAVEBTC')
df.tail(10)

,timestamp,open,high,low,close,volume,buy_volume
36,1734194100000,0.003632,0.003632,0.003629,0.003630,0.893,0.666
37,1734194160000,0.003633,0.003633,0.003628,0.003628,0.361,0.163
38,1734194220000,0.003628,0.003633,0.003627,0.003633,1.905,1.707
39,1734194280000,0.003637,0.003638,0.003633,0.003633,0.661,0.504
40,1734194340000,0.003632,0.003633,0.003630,0.003631,1.133,1.034
41,1734194400000,0.003631,0.003634,0.003631,0.003634,0.194,0.128
42,1734194460000,0.003639,0.003639,0.003631,0.003631,0.180,0.000
43,1734194520000,0.003630,0.003632,0.003630,0.003632,0.428,0.132
44,1734194580000,0.003632,0.003635,0.003632,0.003635,0.372,0.098
45,1734194640000,0.003636,0.003636,0.003636,0.003636,0.028,0.000


In [ ]:
from data.calculate.crypto_binance import calculate_technical_indicators
from data.fetch.crypto_binance import fetch_symbol_list_binance
market_name = 'crypto_binance'
symbol_list = fetch_symbol_list_binance(type='spot', suffix='BTC')
symbol_list = [symbol.replace('/', '') for symbol in symbol_list]
calculate_technical_indicators(market_name=market_name, symbol_list=symbol_list, timeframe=timeframe)

In [ ]:
from finstore.finstore import Finstore
finstore = Finstore(market_name=market_name, timeframe=timeframe, enable_append=True)
symbol_list = finstore.read.get_symbol_list()
df = finstore.read.symbol_list(symbol_list=symbol_list, merged_dataframe=True)
df

In [4]:
for symbol in df:
    df[symbol]['average_volume_90'] = 0
    df[symbol].dropna(inplace=True)

In [ ]:
df['XRPBTC'].tail(10)

In [67]:
import pandas as pd

# Iterate over each symbol in the DataFrame
for symbol in df:
    # Convert the timestamp to a readable datetime format
    df[symbol].index = pd.to_datetime(df[symbol].index, unit='ms')
    
    # Calculate the rolling average volume over the last 14 data points
    df[symbol]['average_volume_14'] = df[symbol]['buy_volume'].rolling(window=14).mean()
    
    # Create a new column to flag if the volume is greater than 5 times the average volume
    df[symbol]['volume_spike'] = (df[symbol]['buy_volume'] > 10 * df[symbol]['average_volume_14']).astype(int)

# Collect all data points where a volume spike is detected
spike_data_points = []
for symbol in df:
    spikes = df[symbol][df[symbol]['volume_spike'] == 1]
    for index, row in spikes.iterrows():
        spike_data_points.append((symbol, index, row['buy_volume']))

# spike_data_points now contains tuples of (symbol, timestamp, volume) for each spike

In [ ]:
# Import the necessary function
from utils.calculation.slope_r2 import calculate_exponential_regression_volume

# Iterate over each symbol in the DataFrame
for symbol in df:
    # Calculate the exponential regression volume for each symbol
    df_calculated = calculate_exponential_regression_volume(df[symbol], window=30)
    
    # Add the calculated indicator as a new column to the original DataFrame
    df[symbol]['slope_r2_volume_30'] = df_calculated['indicator_value']

# Example: Display the last 10 rows of the updated DataFrame for 'ETHBTC'
df['XRPBTC'].tail(10)


In [ ]:
# ... existing code ...

import pandas as pd

# Create an empty DataFrame to store all entries
all_entries = pd.DataFrame()

# Iterate over each symbol in the DataFrame
for symbol in df:
    # Add a column to identify the symbol
    df[symbol]['symbol'] = symbol
    df[symbol]['timestamp'] = pd.to_datetime(df[symbol]['timestamp'], unit='ms')
    
    # Append the DataFrame to the all_entries DataFrame
    all_entries = pd.concat([all_entries, df[symbol]], ignore_index=True)

# Sort the combined DataFrame by the slope_r2_volume_30 column in descending order
all_entries_sorted = all_entries.sort_values(by='slope_r2_volume_30', ascending=False)

# Select the top 10 entries
top_10_entries = all_entries_sorted.head(10)

# Display the top 10 entries
top_10_entries

In [44]:
from strategy.private.crypto_sotm import CryptoSOTM
crypto_sotm = CryptoSOTM()
top_n_symbols = crypto_sotm.get_top_n_symbols(df, n=10, r2_period=30)
telegram_message = crypto_sotm.get_top_n_pairs_message(top_n_symbols, n=10)
telegram_message

'*Top 10 alt/BTC pairs:*\n• *KMDBTC*: `0.000494` (Score: 4/4)\n• *EIGENBTC*: `0.000407` (Score: 4/4)\n• *LDOBTC*: `0.000308` (Score: 4/4)\n• *SCRBTC*: `0.000283` (Score: 4/4)\n• *LITBTC*: `0.000239` (Score: 4/4)\n• *GLMRBTC*: `0.000237` (Score: 4/4)\n• *SUIBTC*: `0.000199` (Score: 4/4)\n• *BANANABTC*: `0.000194` (Score: 4/4)\n• *ZECBTC*: `0.000179` (Score: 4/4)\n• *DOTBTC*: `0.000168` (Score: 4/4)'